In [16]:
import cv2
import os

# Directory to save the detected faces
output_dir = 'detected_faces'
os.makedirs(output_dir, exist_ok=True)

In [17]:
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub
from PIL import Image

In [21]:
# Load FaceNet model from TensorFlow Hub
model_url = "https://tfhub.dev/google/imagenet/inception_resnet_v1_face/feature_vector/1"
embedder = hub.load(model_url)

OSError: https://tfhub.dev/google/imagenet/inception_resnet_v1_face/feature_vector/1 does not appear to be a valid module.

In [ ]:

# Pre-process images and extract embeddings
def extract_embedding(image_path):
    img = Image.open(image_path).convert("RGB")  # Convert to RGB if necessary
    img = img.resize((160, 160))  # Resize image to match FaceNet input size
    img = np.array(img) / 255.0  # Normalize pixel values
    img = tf.expand_dims(img, axis=0)  # Add batch dimension
    embedding = embedder(img)[0]  # Extract embeddings
    return embedding.numpy()


In [ ]:

# List of image paths
image_paths = os.listdir(path="./dataset/guru")[8:30]
image_paths

In [ ]:
# List to store embeddings for each image
embeddings = []

# Extract embeddings for each image
for img_path in image_paths:
    embedding = extract_embedding(img_path)
    embeddings.append(embedding)

# Aggregate embeddings
aggregate_embedding = np.mean(embeddings, axis=0)

# Now you have the aggregated embedding representing the person


In [ ]:
# Load the pre-trained face detection classifier
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
# Start capturing video from the default camera (0)
video_capture = cv2.VideoCapture(0)

while True:
    # Capture frame-by-frame
    ret, frame = video_capture.read()

    # Convert the frame to grayscale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Detect faces in the grayscale frame
    faces = face_cascade.detectMultiScale(
        gray,
        scaleFactor=1.1,
        minNeighbors=5,
        minSize=(100, 100),
        flags=cv2.CASCADE_SCALE_IMAGE
    )

    # Draw a rectangle around the detected faces
    for (x, y, w, h) in faces:
        cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)

        # Crop the region of interest (ROI) where the face is detected
        face_roi = frame[y:y+h, x:x+w]
        
         # Save the detected face to the output directory
        filename = os.path.join(output_dir, f'face_{x}_{y}.jpg')
        cv2.imwrite(filename, face_roi)
    # Display the resulting frame
    cv2.imshow('Video', frame)

    # Break the loop when 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the video capture object and close all windows
video_capture.release()
cv2.destroyAllWindows()